<a href="https://colab.research.google.com/github/JerryChenz/Stock_screener/blob/master/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

class InitalScreen:
  '''Inital Screening of the Venture'''

  def __init__(self, investment_size, sector, location, stage):
    self.investment_size = investment_size
    self.sector = sector
    self.location = location
    self.stage = stage

  @property
  def size_score(self):
    if investment_size < 500 or investment_size > 100:
      inital_score = inital_score + 1
      print(inital_score)


  def x__init__(self, size_score, sector_score, location_score, maturity_score):
    self.size_score = size_score
    self.sector_score = sector_score
    self.location_score = location_score
    self.maturity_score = maturity_score
    self.total = self.size_score + self.sector_score + self.location_score + self.maturity_score

In [ ]:
inital_score = 0
investment_size = 400 #@param {type:"number"}

if investment_size < 500 or investment_size > 100:
  print('ok')
  inital_score = inital_score + 1
  print(inital_score)